In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import eli5
import time

from sklearn.preprocessing import LabelEncoder # for encoding
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler #for standardization
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, plot_confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_curve, roc_curve
from eli5.sklearn import PermutationImportance # 피처 중요도 평가
from collections import Counter # 이상치 제거
from sklearn.model_selection import GridSearchCV # 최적 파라미터 
from imblearn.over_sampling import SMOTE # 오버샘플링

/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [20]:
def preprocessing(df_scaling, scaled_form = 'MinMaxScaler()'):
    # 불필요한 컬럼 제거
    if 'ID' in df_scaling:
        df_scaling = df_scaling.drop("ID", axis = 1)
        if len(df_scaling.columns) == 1:
            return df_scaling
        # 시력(eyesight) 데이터 범주화
        # 시력 기준
        # (0.1~0.9): 나쁨(1)
        # (1.0~1.5): 보통(2)
        # (1.6~2.0): 좋음(3)
        # (9.9): 실명(4)

        def func(x):
            if x < 1.0 :
                return 1
            elif x < 1.6 :
                return 2
            elif x <= 2.0 :
                return 3
            else : 
                return 4
        df_scaling['eyesight(left)'] = df_scaling['eyesight(left)'].apply(lambda x:func(x))    
        df_scaling['eyesight(right)'] = df_scaling['eyesight(right)'].apply(lambda x:func(x))
        
        
        df_scaling = df_scaling.drop('oral', axis = 1) 
        # 범주형 피처 레이블 인코딩 
        cate_features = df_scaling[['gender','tartar']]

        lbe = LabelEncoder()
        lbe.fit_transform(df_scaling["gender"])
        df_scaling["gender"] = lbe.fit_transform(df_scaling["gender"])

        lbe = LabelEncoder()
        lbe.fit_transform(df_scaling["tartar"])
        df_scaling["tartar"] = lbe.fit_transform(df_scaling["tartar"])
        # hearing 피처 1, 2 => 1, 0으로 변환
        df_scaling['hearing(left)'] = df_scaling['hearing(left)'].apply(lambda x: x-2 if x ==2.0 else x )
        df_scaling['hearing(right)'] = df_scaling['hearing(right)'].apply(lambda x: x-2 if x ==2.0 else x )
        
        # Urine protein categorizing
        x = df_scaling['Urine protein']
        for i in range(len(x)):
            if(x[i] == 1.0):
                x[i] = 0
            elif(x[i] == 2.0):
                x[i] = 1
            else:
                x[i] = 2
        df_scaling['Urine protein'] = x
        
#         # Urine protein 범주화
#         df_scaling['Urine protein'] = df_scaling['Urine protein'].apply(lambda x : x==0 if x == 1.0 else x)
#         df_scaling['Urine protein'] = df_scaling['Urine protein'].apply(lambda x : x==1 if x == 2.0 else x)
#         df_scaling['Urine protein'] = df_scaling['Urine protein'].apply(lambda x : x==2 if x == 3.0 else x)
#         df_scaling['Urine protein'] = df_scaling['Urine protein'].apply(lambda x : x==2 if x > 3.0 else x)


        # BMI 지수 계산
        # bmi = kg/m^2
        df_scaling['bmi'] = df_scaling['weight(kg)']/((df_scaling['height(cm)']*0.01)**2)
        # wwi(비만 지수) 지수 계산
        df_scaling['wwi'] = df_scaling['waist(cm)']/(df_scaling['weight(kg)'].apply(np.sqrt))
        return df_scaling

In [5]:
def scaling(train_data, test_data, scaled_form = 'MinMaxScaler()'):
    # 형태 별 특성 구분
    train_data = preprocessing(train_data)
    test_data = preprocessing(test_data)
    tr_cate_features = train_data[['gender', 'tartar', 'hearing(right)', 'hearing(left)', 'dental caries']]
    tr_scaled_features = train_data.drop(tr_cate_features.columns, axis=1)
    
    ts_cate_features = test_data[['gender', 'tartar', 'hearing(right)', 'hearing(left)', 'dental caries']]
    ts_scaled_features = test_data.drop(ts_cate_features.columns, axis=1)
    
    if scaled_form == 'StandardScaler()':
        # Standard scaler
        scaler = StandardScaler()
        scaler.fit(tr_scaled_features) # 훈련 데이터에 fit() 적용
        
        # 훈련 데이터와 테스트 데이터에 transform()을 통해 변환
        tr_scaled = scaler.transform(tr_scaled_features)
        ts_scaled = scaler.transform(ts_scaled_features)
        
        train_std_scaled = pd.DataFrame(tr_scaled, columns=tr_scaled_features.columns)
        train_std_scaled[tr_cate_features.columns] = tr_cate_features
        
        test_std_scaled = pd.DataFrame(ts_scaled, columns=ts_scaled_features.columns)
        test_std_scaled[ts_cate_features.columns] = ts_cate_features
        
        return train_std_scaled, test_std_scaled
    
    elif scaled_form == 'RobustScaler()':
        # Robust scaler
        scaler = RobustScaler()
        scaler.fit(tr_scaled_features) # 훈련 데이터에 fit() 적용
        
        # 훈련 데이터와 테스트 데이터에 transform()을 통해 변환
        tr_scaled = scaler.transform(tr_scaled_features)
        ts_scaled = scaler.transform(ts_scaled_features)
        
        # 데이터 프레임 형태로 변환
        train_robust_scaled = pd.DataFrame(tr_scaled, columns=tr_scaled_features.columns)
        train_robust_scaled[tr_cate_features.columns] = tr_cate_features
        
        test_robust_scaled = pd.DataFrame(ts_scaled, columns=ts_scaled_features.columns)
        test_robust_scaled[ts_cate_features.columns] = ts_cate_features
        
        return train_robust_scaled, test_robust_scaled
    
    elif scaled_form == 'logScaler' :
        tr_scaled = np.log1p(tr_scaled_features)
        ts_scaled = np.log1p(ts_scaled_features)
         # 데이터 프레임 형태로 변환
        train_log_scaled = pd.DataFrame(tr_scaled, columns = tr_scaled_features.columns)
        train_log_scaled[tr_cate_features.columns] = tr_cate_features
        
        test_log_scaled = pd.DataFrame(ts_scaled,columns = ts_scaled_features.columns)
        test_log_scaled[ts_cate_features.columns] = ts_cate_features 
        
        return train_log_scaled, test_log_scaled
        
    else:
        # MinMax scaler
        scaler = MinMaxScaler()
        scaler.fit(tr_scaled_features) # 훈련 데이터에 fit() 적용
        
        # 훈련 데이터와 테스트 데이터에 transform()을 통해 변환
        tr_scaled = scaler.transform(tr_scaled_features)
        ts_scaled = scaler.transform(ts_scaled_features)
        
        # 데이터 프레임 형태로 변환
        train_mmx_scaled = pd.DataFrame(tr_scaled, columns = tr_scaled_features.columns)
        train_mmx_scaled[tr_cate_features.columns] = tr_cate_features
        
        test_mmx_scaled = pd.DataFrame(ts_scaled,columns = ts_scaled_features.columns)
        test_mmx_scaled[ts_cate_features.columns] = ts_cate_features

        return train_mmx_scaled, test_mmx_scaled

In [6]:
# 평가 지표 함수
def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [7]:
# 이상치 제거 함수
def get_outlier(df=None, column=None, weight=1.5):
    quantile_25 = np.percentile(df[column].values, 25)
    quantile_75 = np.percentile(df[column].values, 75)

    IQR = quantile_75 - quantile_25
    IQR_weight = IQR*weight

    lowest = quantile_25 - IQR_weight
    highest = quantile_75 + IQR_weight

    outlier_idx = df[column][ (df[column] < lowest) | (df[column] > highest) ].index
    return outlier_idx

In [8]:
# 중요도 평가 함수
def feature_importance():
    lr_clf = LogisticRegression(solver = 'liblinear', C=11, penalty='l1', max_iter = 1000)
    lr_clf.fit(x_train, y_train)
    lr_pred = lr_clf.predict(x_test)
    coef = pd.DataFrame(lr_clf.coef_.reshape(-1,1), index = x_train.columns,columns=['coef'])
    coef = coef.apply(lambda x : abs(x)).sort_values(by=['coef'],ascending=False)
    return coef

### 로지스틱 회귀
- 선형회귀를 기반, 시그모이드 함수를 이용해 분류를 수행함
- 주요 파라미터
    - solver : ibfgs(기본값), liblinear, newton-cg, sag, saga
    - penalty : l1, l2 
    - C(CostFunction) :  규제 강도
- 함수 정리
    - 전처리 함수 : preprocessing(df)
        - 불필요 컬럼 제거, 레이블 인코딩, BMI, WWI지수 계산 후 가공컬럼으로 삽입
        
    - 스케일 함수 : scaling(train, test, 'scaling method')
        - standard scaler, min max scaler, robust scaler로 선택, 스케일 진행
        
    - 평가지표 함수 : get_clf_eval(y_test, pred=None, pred_proba=None)
        - 정확도, 정밀도, 재현율, f1 score, AUC score 계산
        
    - 이상치 제거함수 : get_outlier(df=None, column=None, weight=1.5)
        - IQR rull로 각 컬럼에 이상치를 제거
        
    - 피처 중요도 평가 함수 : feature_importance()
        - 상관도별 중요도 평가

#### 1. 기본 모델링
- 불필요 컬럼 제거, 범주형 피처 레이블 인코딩, BMI, WWI 컬럼 생성
    - 기본 Logistic 모델 성능
        - 정확도: 0.7277, 정밀도: 0.6499, 재현율: 0.5652, F1: 0.6046, AUC:0.8126 
        - time : 0.3336019515991211
    - solver 값을 변경해가며 적용했으나 성능의 큰 차이는 없음

In [156]:
# 연속형 변수에 로그변환 적용
# gender -> 원핫 인코딩
# Urine protein 카테고리형 - 0, 1, 2 
# 각자 한 모델 정리

# 각 모델별 정리
# 하이퍼 파라미터에 대한 정리
# 

In [141]:
# 1.1 파일 로드
x_train = pd.read_csv('./data/Smoking_raw/competition_format/x_train.csv')
x_test = pd.read_csv('./data/Smoking_raw/competition_format/x_test.csv')
y_train = pd.read_csv('./data/Smoking_raw/competition_format/y_train.csv')
y_test = pd.read_csv('./data/Smoking_raw/competition_format/y_test.csv')
# 1.2 전처리 - 불필요 컬럼 제거, 범주형 피처 레이블 인코딩, bmi 컬럼 생성, 정규화작업   
x_train = preprocessing(x_train)
x_test = preprocessing(x_test)
y_train = preprocessing(y_train)
y_test = preprocessing(y_test)

In [142]:
start = time.time()
# 1.3 학습/예측
lr_clf = LogisticRegression()
lr_clf.fit(x_train, y_train)
lr_pred = lr_clf.predict(x_test)
pred_prob = lr_clf.predict_proba(x_test)[:,1]

# 1.4 평가
get_clf_eval(y_test,lr_pred, pred_prob)

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간


/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


오차 행렬
[[5787 1249]
 [1784 2319]]
정확도: 0.7277, 정밀도: 0.6499, 재현율: 0.5652,    F1: 0.6046, AUC:0.8126
time : 0.3648698329925537


/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### 2. 최적 하이퍼파라미터 탐색
- 스케일러 적용
- 이상치 제거
- 최적 하이퍼파라미터 탐색 후 모델링 진행
    - 최적 하이퍼 파라미터
        - 최적 파라미터 : {'C': 11, 'penalty': 'l1', 'solver': 'liblinear'}, 최적 평균 정확도 0.762
        - time : 151.5772349834442
    - 모델링 평가 점수
        - 정확도: 0.7488, 정밀도: 0.6486, 재현율: 0.6941,    F1: 0.6706, AUC:0.8338
        - time : 4.220590114593506

In [132]:
# 2.1 파일 로드
x_train = pd.read_csv('./data/Smoking_raw/competition_format/x_train.csv')
x_test = pd.read_csv('./data/Smoking_raw/competition_format/x_test.csv')
y_train = pd.read_csv('./data/Smoking_raw/competition_format/y_train.csv')
y_test = pd.read_csv('./data/Smoking_raw/competition_format/y_test.csv')
# 2.2 전처리 - 불필요 컬럼 제거, 범주형 피처 레이블 인코딩, bmi 컬럼 생성, 정규화작업
scaled = scaling(x_train, x_test)
x_train = pd.DataFrame(scaled[0])
x_test = pd.DataFrame(scaled[1])
y_train = preprocessing(y_train)
y_test = preprocessing(y_test)

In [133]:
# 2.3 이상치 제거
for i in x_train.columns:
    outlier_idx = get_outlier(df=x_train, column=i, weight=1.5)
    x_train.drop(outlier_idx, axis=0, inplace=True)
    y_train.drop(outlier_idx, axis=0, inplace=True)
for i in x_test.columns:
    outlier_idx = get_outlier(df=x_test, column=i, weight=1.5)
    x_test.drop(outlier_idx, axis=0, inplace=True)
    y_test.drop(outlier_idx, axis=0, inplace=True)

In [134]:
# 2.4 최적파라미터 탐색 
start = time.time()

params = {'solver' : ['liblinear', 'lbfgs', 'sag','saga','newton-cg'],
         'penalty' : ['l2','l1'],
         'C':[9, 10, 11, 11.3, 11.5, 11.8, 12, 13]}
lr_clf = LogisticRegression()
grid_clf = GridSearchCV(lr_clf, param_grid = params, scoring = 'accuracy', cv = 5)
grid_clf.fit(x_train, y_train)
print('최적 파라미터 : {0}, 최적 평균 정확도 {1:.3f}'.format(grid_clf.best_params_, grid_clf.best_score_))

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d

/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/

/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS R

/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS R

...

/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1)

/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/

/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS R

/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d

최적 파라미터 : {'C': 11, 'penalty': 'l1', 'solver': 'liblinear'}, 최적 평균 정확도 0.763
time : 144.32885599136353


In [143]:
start = time.time()
# 2.5 학습/예측 (최적 하이퍼파라미터로 학습)
lr_clf = LogisticRegression(C=11, penalty= 'l1', solver = 'liblinear')
lr_clf.fit(x_train, y_train)
lr_pred = lr_clf.predict(x_test)
pred_prob = lr_clf.predict_proba(x_test)[:,1]

# 2.6 평가
get_clf_eval(y_test,lr_pred, pred_prob)

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


오차 행렬
[[5493 1543]
 [1255 2848]]
정확도: 0.7488, 정밀도: 0.6486, 재현율: 0.6941,    F1: 0.6706, AUC:0.8338
time : 4.220590114593506


#### 3. oversampling 적용
- 스케일러 적용
- 이상치 제거
- 최적 하이퍼파라미터 적용
- oversampling 적용
    - 정확도: 0.7283, 정밀도: 0.5354, 재현율: 0.8929,    F1: 0.6694, AUC:0.8364
        - 재현율이 크게 개선됨
        - 정밀도와 재현율의 비율이 좋지 못함

In [144]:
# 3.1 파일 로드
x_train = pd.read_csv('./data/Smoking_raw/competition_format/x_train.csv')
x_test = pd.read_csv('./data/Smoking_raw/competition_format/x_test.csv')
y_train = pd.read_csv('./data/Smoking_raw/competition_format/y_train.csv')
y_test = pd.read_csv('./data/Smoking_raw/competition_format/y_test.csv')
# 3.2 전처리 - 불필요 컬럼 제거, 범주형 피처 레이블 인코딩, bmi 컬럼 생성, 정규화작업   
scaled = scaling(x_train, x_test)
x_train = pd.DataFrame(scaled[0])
x_test = pd.DataFrame(scaled[1])
y_train = preprocessing(y_train)
y_test = preprocessing(y_test)

In [145]:
# 3.3 이상치 삭제
for i in x_train.columns:
    outlier_idx = get_outlier(df=x_train, column=i, weight=1.5)
    x_train.drop(outlier_idx, axis=0, inplace=True)
    y_train.drop(outlier_idx, axis=0, inplace=True)
for i in x_test.columns:
    outlier_idx = get_outlier(df=x_test, column=i, weight=1.5)
    x_test.drop(outlier_idx, axis=0, inplace=True)
    y_test.drop(outlier_idx, axis=0, inplace=True)

In [146]:
# 3.4 oversampling
smote = SMOTE(random_state = 0)
x_train_over, y_train_over = smote.fit_resample(x_train, y_train)

print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', x_train.shape, y_train.shape)
print()
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', x_train_over.shape, y_train_over.shape)
print('SMOTE 적용 후 레이블 값 분포: \n',y_train_over.value_counts())

SMOTE 적용 전 학습용 피처/레이블 데이터 세트:  (19435, 26) (19435, 1)

SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (27264, 26) (27264, 1)
SMOTE 적용 후 레이블 값 분포: 
 smoking
0          13632
1          13632
dtype: int64


In [148]:
# 3.5 학습/예측
start = time.time()

lr_clf = LogisticRegression(C=11, penalty='l1', solver='liblinear', max_iter=1000)
lr_clf.fit(x_train_over, y_train_over)
lr_pred = lr_clf.predict(x_test)
pred_prob = lr_clf.predict_proba(x_test)[:,1]

# 3.6 평가
get_clf_eval(y_test,lr_pred, pred_prob)

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


오차 행렬
[[2182 1151]
 [ 158 1326]]
정확도: 0.7283, 정밀도: 0.5353, 재현율: 0.8935,    F1: 0.6695, AUC:0.8364
time : 8.898006677627563


#### 4. 피처 중요도 평가
- 낮은 순위의 피처들을 삭제 후 모델링
- 10개에서 1개까지 순차 삭제 후 모델링
- 스케일러 적용
- 이상치 제거 미적용
    - 이상치 제거 후 피처삭제는 재현율이 현저히 떨어짐
- 최적 하이퍼파라미터 적용
    - 삭제 수 :  -2  열 갯수 :  24
    - 정확도: 0.7490, 정밀도: 0.6486, 재현율: 0.6951,    F1: 0.6711, AUC:0.8338

In [149]:
# 이상치함수 적용
# for 문을 이용, 상관도가 낮은 피쳐 삭제 후 평가
start = time.time()

for i in range(-10,0):
    # 파일 로드
    x_train = pd.read_csv('./data/Smoking_raw/competition_format/x_train.csv')
    x_test = pd.read_csv('./data/Smoking_raw/competition_format/x_test.csv')
    y_train = pd.read_csv('./data/Smoking_raw/competition_format/y_train.csv')
    y_test = pd.read_csv('./data/Smoking_raw/competition_format/y_test.csv')
    # 전처리 - 불필요 컬럼 제거, 범주형 피처 레이블 인코딩, bmi 컬럼 생성, 정규화작업   
    scaled = scaling(x_train, x_test)
    x_train = pd.DataFrame(scaled[0])
    x_test = pd.DataFrame(scaled[1])
    y_train = preprocessing(y_train)
    y_test = preprocessing(y_test)
    
#     cat_features = ['gender', 'tartar', 'hearing(right)', 'hearing(left)', 'dental caries']
#     scaled_features = x_train.drop(cat_features, axis=1)

#     # 함수 사용해서 이상치 값 삭제
#     for k in scaled_features.columns:
#         outlier_idx = get_outlier(df=x_train, column=k, weight=1.5)
#         x_train.drop(outlier_idx, axis=0, inplace=True)
#         y_train.drop(outlier_idx, axis=0, inplace=True)
#     for j in scaled_features.columns:
#         outlier_idx = get_outlier(df=x_test, column=j, weight=1.5)
#         x_test.drop(outlier_idx, axis=0, inplace=True)
#         y_test.drop(outlier_idx, axis=0, inplace=True)

    # 상관도가 낮은 feature name
     # 중요도 평가 후 낮은 순위 9개 삭제   
    coef = feature_importance()
    low_coef_features = coef[i:].index 
    x_train.drop(low_coef_features,axis=1,inplace=True)
    x_test.drop(low_coef_features,axis=1,inplace=True)
    
    # 로지스틱 회귀 - (C=11.3, penalty='l1', solver='liblinear', max_iter=1000)
    lr_clf = LogisticRegression(C=11, penalty='l1', solver='liblinear', max_iter=1000)
    lr_clf.fit(x_train, y_train)
    lr_pred = lr_clf.predict(x_test)
    pred_prob = (lr_clf.predict_proba(x_test)[:,1])
    # 평가
    print('삭제 수 : ',i,' 열 갯수 : ',len(x_train.columns))
    get_clf_eval(y_test,lr_pred, pred_prob)
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


삭제 수 :  -10  열 갯수 :  16
오차 행렬
[[5490 1546]
 [1262 2841]]
정확도: 0.7479, 정밀도: 0.6476, 재현율: 0.6924,    F1: 0.6693, AUC:0.8339


/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


삭제 수 :  -9  열 갯수 :  17
오차 행렬
[[5490 1546]
 [1261 2842]]
정확도: 0.7480, 정밀도: 0.6477, 재현율: 0.6927,    F1: 0.6694, AUC:0.8339


/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

삭제 수 :  -3  열 갯수 :  23
오차 행렬
[[5493 1543]
 [1254 2849]]
정확도: 0.7489, 정밀도: 0.6487, 재현율: 0.6944,    F1: 0.6707, AUC:0.8338


/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


삭제 수 :  -2  열 갯수 :  24
오차 행렬
[[5491 1545]
 [1250 2853]]
정확도: 0.7491, 정밀도: 0.6487, 재현율: 0.6953,    F1: 0.6712, AUC:0.8338


/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


삭제 수 :  -1  열 갯수 :  25
오차 행렬
[[5496 1540]
 [1250 2853]]
정확도: 0.7495, 정밀도: 0.6494, 재현율: 0.6953,    F1: 0.6716, AUC:0.8338
time : 235.19370102882385


#### 5. 피처 중요도 순으로 이상치 제거함수 적용
- 높은 순위별로 이상치 제거함수를 적용하며 학습/예측/평가
- 스케일러 적용
- 최적 하이퍼파라미터 적용
    - 이상치 제거함수 적용 컬럼수가 많아질수록 정확도가 개선됨
    - 반대로 재현율은 떨어지는 경향을 보임
    - 정확도와 재현율이 가장 높은 모델을 선택함
        - 이상치 제거함수 적용 수 :  1  행 갯수 :  41432
        - 정확도: 0.7552, 정밀도: 0.6511, 재현율: 0.6802,    F1: 0.6653, AUC:0.8410

In [150]:
# 이상치 제거함수 순차 적용
# 최적하이퍼파라미터 적용
start = time.time()

for i in range(len(x_train.columns)):
    x_train = pd.read_csv('./data/Smoking_raw/competition_format/x_train.csv')
    x_test = pd.read_csv('./data/Smoking_raw/competition_format/x_test.csv')
    y_train = pd.read_csv('./data/Smoking_raw/competition_format/y_train.csv')
    y_test = pd.read_csv('./data/Smoking_raw/competition_format/y_test.csv')
    # 전처리 - 불필요 컬럼 제거, 범주형 피처 레이블 인코딩, bmi 컬럼 생성, 정규화작업   
    scaled = scaling(x_train, x_test)
    x_train = pd.DataFrame(scaled[0])
    x_test = pd.DataFrame(scaled[1])
    y_train = preprocessing(y_train)
    y_test = preprocessing(y_test)
    
    coef = feature_importance()
    high_coef_features = coef[:i].index 
    
    # 함수 사용해서 이상치 값 삭제
    # 중요도가 높은 순으로 n개의 컬럼에 이상치 제거 함수 적용해보기
    for k in high_coef_features:
        outlier_idx = get_outlier(df=x_train, column=k, weight=1.5)
        x_train.drop(outlier_idx, axis=0, inplace=True)
        y_train.drop(outlier_idx, axis=0, inplace=True)
    for j in high_coef_features:
        outlier_idx = get_outlier(df=x_test, column=j, weight=1.5)
        x_test.drop(outlier_idx, axis=0, inplace=True)
        y_test.drop(outlier_idx, axis=0, inplace=True)

    lr_clf = LogisticRegression(C=11, penalty='l1', solver='liblinear', max_iter=1000)
    lr_clf.fit(x_train, y_train)
    lr_pred = lr_clf.predict(x_test)
    pred_prob = (lr_clf.predict_proba(x_test)[:,1])
    # 평가
    print('이상치 제거함수 적용 수 : ',i,' 행 갯수 : ',len(x_train.index))
    get_clf_eval(y_test,lr_pred, pred_prob)
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


이상치 제거함수 적용 수 :  0  행 갯수 :  44553
오차 행렬
[[5495 1541]
 [1253 2850]]
정확도: 0.7492, 정밀도: 0.6491, 재현율: 0.6946,    F1: 0.6711, AUC:0.8338


/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


이상치 제거함수 적용 수 :  1  행 갯수 :  41432
오차 행렬
[[5310 1353]
 [1186 2526]]
정확도: 0.7553, 정밀도: 0.6512, 재현율: 0.6805,    F1: 0.6655, AUC:0.8410


/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

이상치 제거함수 적용 수 :  22  행 갯수 :  22138
오차 행렬
[[3395  509]
 [ 797  848]]
정확도: 0.7646, 정밀도: 0.6249, 재현율: 0.5155,    F1: 0.5650, AUC:0.8408


/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


이상치 제거함수 적용 수 :  23  행 갯수 :  21809
오차 행렬
[[3341  502]
 [ 787  833]]
정확도: 0.7640, 정밀도: 0.6240, 재현율: 0.5142,    F1: 0.5638, AUC:0.8395


/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


이상치 제거함수 적용 수 :  24  행 갯수 :  19728
오차 행렬
[[3023  475]
 [ 680  768]]
정확도: 0.7665, 정밀도: 0.6179, 재현율: 0.5304,    F1: 0.5708, AUC:0.8419
time : 558.8864178657532


#### 6. 중요도 평가에서 낮은 점수를 받은 피처를 삭제한 후 이상치 제거함수 순차 적용
- 피처 중요도가 낮은 2개의 피처 삭제
- 스케일러 적용
- 최적 하이퍼파라미터 적용
    - 현재 컬럼 수 :  24 이상치 제거함수 적용 수 :  1  행 갯수 :  41432
    - 정확도: 0.7564, 정밀도: 0.6526, 재현율: 0.6827,    F1: 0.6673, AUC:0.8410

In [151]:
start = time.time()

# 이상치 제거함수 순차 적용
# 최적하이퍼파라미터 적용
for i in range(len(x_train.columns)):
    x_train = pd.read_csv('./data/Smoking_raw/competition_format/x_train.csv')
    x_test = pd.read_csv('./data/Smoking_raw/competition_format/x_test.csv')
    y_train = pd.read_csv('./data/Smoking_raw/competition_format/y_train.csv')
    y_test = pd.read_csv('./data/Smoking_raw/competition_format/y_test.csv')
    # 전처리 - 불필요 컬럼 제거, 범주형 피처 레이블 인코딩, bmi 컬럼 생성, 정규화작업   
    scaled = scaling(x_train, x_test)
    x_train = pd.DataFrame(scaled[0])
    x_test = pd.DataFrame(scaled[1])
    y_train = preprocessing(y_train)
    y_test = preprocessing(y_test)
    
    coef = feature_importance()
    high_coef_features = coef[:i].index 
    
    # 가장 낮은 상관도를 보인 2개 컬럼 삭제
    x_train.drop(coef[-2:].index,axis=1,inplace=True)
    x_test.drop(coef[-2:].index,axis=1,inplace=True)
    
    # 함수 사용해서 이상치 값 삭제
    # 중요도가 높은 순으로 n개의 컬럼에 이상치 제거 함수 적용해보기
    for k in high_coef_features:
        outlier_idx = get_outlier(df=x_train, column=k, weight=1.5)
        x_train.drop(outlier_idx, axis=0, inplace=True)
        y_train.drop(outlier_idx, axis=0, inplace=True)
    for j in high_coef_features:
        outlier_idx = get_outlier(df=x_test, column=j, weight=1.5)
        x_test.drop(outlier_idx, axis=0, inplace=True)
        y_test.drop(outlier_idx, axis=0, inplace=True)

    lr_clf = LogisticRegression(C=11, penalty='l1', solver='liblinear', max_iter=1000)
    lr_clf.fit(x_train, y_train)
    lr_pred = lr_clf.predict(x_test)
    pred_prob = (lr_clf.predict_proba(x_test)[:,1])
    # 평가
    print('현재 컬럼 수 : ',len(x_train.columns),'이상치 제거함수 적용 수 : ',i,' 행 갯수 : ',len(x_train.index))
    get_clf_eval(y_test,lr_pred, pred_prob)
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


현재 컬럼 수 :  24 이상치 제거함수 적용 수 :  0  행 갯수 :  44553
오차 행렬
[[5492 1544]
 [1253 2850]]
정확도: 0.7489, 정밀도: 0.6486, 재현율: 0.6946,    F1: 0.6708, AUC:0.8338


/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


현재 컬럼 수 :  24 이상치 제거함수 적용 수 :  1  행 갯수 :  41432
오차 행렬
[[5313 1350]
 [1184 2528]]
정확도: 0.7558, 정밀도: 0.6519, 재현율: 0.6810,    F1: 0.6661, AUC:0.8412


/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

현재 컬럼 수 :  24 이상치 제거함수 적용 수 :  23  행 갯수 :  20854
오차 행렬
[[3175  493]
 [ 740  810]]
정확도: 0.7637, 정밀도: 0.6216, 재현율: 0.5226,    F1: 0.5678, AUC:0.8380


/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


현재 컬럼 수 :  24 이상치 제거함수 적용 수 :  24  행 갯수 :  20823
오차 행렬
[[3168  496]
 [ 740  804]]
정확도: 0.7627, 정밀도: 0.6185, 재현율: 0.5207,    F1: 0.5654, AUC:0.8375


/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


KeyError: 'age'

### 피드백 결과
#### 연속형 변수에 로그변환 적용 - 스케일러 변경 후 모델링
#### Urine protein 범주화 - 0, 1, 2 

7. 스케일러 변경 후 모델링
- min_max_scaler()
 - 이상치 제거 적용 컬럼수 : 중요도 상위 1개 컬럼
 - 삭제 컬럼 수 : 중요도 하위 2개 컬럼
 - min_max_scaler() 적용
 - 최적 하이퍼파라미터 적용
     - 현재 컬럼 수 :  24 이상치 제거함수 적용 수 :  1  행 갯수 :  10375
     - 정확도: 0.7529, 정밀도: 0.6468, 재현율: 0.6813,    F1: 0.6636, AUC:0.8390
     - time : 11.05785083770752
- standard_scaler()
 - 이상치 제거 적용 컬럼수 : 중요도 상위 1개 컬럼
 - 삭제 컬럼 수 : 중요도 하위 2개 컬럼
 - standard_scaler() 적용
 - 최적 하이퍼파라미터 적용
     - 현재 컬럼 수 :  24 이상치 제거함수 적용 수 :  1  행 갯수 :  10408
     - 정확도: 0.7531, 정밀도: 0.6470, 재현율: 0.6835,    F1: 0.6648, AUC:0.8388
     - time : 12.837147951126099
- logscaler()
 - 이상치 제거 적용 컬럼수 : 중요도 상위 1개 컬럼
 - 삭제 컬럼 수 : 중요도 하위 2개 컬럼
 - log_scaler() 적용
 - 최적 하이퍼파라미터 적용
     - 현재 컬럼 수 :  24 이상치 제거함수 적용 수 :  1  행 갯수 :  10921
     - 정확도: 0.7586, 정밀도: 0.6614, 재현율: 0.6974,    F1: 0.6789, AUC:0.8429
     - time : 35.78000497817993

In [204]:
start = time.time()
scaler = ['MinMaxScaler()','StandardScaler()','logScaler']

# - 이상치 제거 적용 컬럼수 : 중요도 상위 1개 컬럼
# - 삭제 컬럼 수 : 중요도 하위 2개 컬럼
# - scaler 순차적용
# - 최적 하이퍼파라미터 적용
for i in scaler:
    x_train = pd.read_csv('./data/Smoking_raw/competition_format/x_train.csv')
    x_test = pd.read_csv('./data/Smoking_raw/competition_format/x_test.csv')
    y_train = pd.read_csv('./data/Smoking_raw/competition_format/y_train.csv')
    y_test = pd.read_csv('./data/Smoking_raw/competition_format/y_test.csv')
    # 전처리 - 불필요 컬럼 제거, 범주형 피처 레이블 인코딩, bmi 컬럼 생성, 정규화작업   
    scaled = scaling(x_train, x_test,i)
    x_train = pd.DataFrame(scaled[0])
    x_test = pd.DataFrame(scaled[1])
    y_train = preprocessing(y_train)
    y_test = preprocessing(y_test)


    # 가장 낮은 상관도를 보인 1개 컬럼 삭제
    x_train.drop(coef[-2:].index,axis=1,inplace=True)
    x_test.drop(coef[-2:].index,axis=1,inplace=True)

    # 함수 사용해서 이상치 값 삭제
    # 중요도가 높은 순으로 n개의 컬럼에 이상치 제거 함수 적용해보기
    for k in high_coef_features:
        outlier_idx = get_outlier(df=x_train, column=k, weight=1.5)
        x_train.drop(outlier_idx, axis=0, inplace=True)
        y_train.drop(outlier_idx, axis=0, inplace=True)
    for j in high_coef_features:
        outlier_idx = get_outlier(df=x_test, column=j, weight=1.5)
        x_test.drop(outlier_idx, axis=0, inplace=True)
        y_test.drop(outlier_idx, axis=0, inplace=True)

    lr_clf = LogisticRegression(C=11, penalty='l1', solver='liblinear', max_iter=1000)
    lr_clf.fit(x_train, y_train)
    lr_pred = lr_clf.predict(x_test)
    pred_prob = (lr_clf.predict_proba(x_test)[:,1])
    # 평가
    print('scaler : ',i)
    print('현재 컬럼 수 : ',len(x_train.columns),'이상치 제거함수 적용 수 : ',len(high_coef_features),' 행 갯수 : ',len(x_test.index))
    get_clf_eval(y_test,lr_pred, pred_prob)
    print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


scaler :  MinMaxScaler()
현재 컬럼 수 :  24 이상치 제거함수 적용 수 :  1  행 갯수 :  10375
오차 행렬
[[5282 1381]
 [1183 2529]]
정확도: 0.7529, 정밀도: 0.6468, 재현율: 0.6813,    F1: 0.6636, AUC:0.8390
time : 11.05785083770752


/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


scaler :  StandardScaler()
현재 컬럼 수 :  24 이상치 제거함수 적용 수 :  1  행 갯수 :  10408
오차 행렬
[[5290 1390]
 [1180 2548]]
정확도: 0.7531, 정밀도: 0.6470, 재현율: 0.6835,    F1: 0.6648, AUC:0.8388
time : 12.837147951126099


/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


scaler :  logScaler
현재 컬럼 수 :  24 이상치 제거함수 적용 수 :  1  행 갯수 :  10921
오차 행렬
[[5498 1427]
 [1209 2787]]
정확도: 0.7586, 정밀도: 0.6614, 재현율: 0.6974,    F1: 0.6789, AUC:0.8429
time : 35.78000497817993


8. 최종 모델링
- 이상치 제거 적용 컬럼수 : 중요도 상위 1개 컬럼
- 삭제 컬럼 수 : 중요도 하위 2개 컬럼
- 스케일러 적용
- 최적 하이퍼파라미터 적용
    - 현재 컬럼 수 :  24 이상치 제거함수 적용 수 :  1  행 갯수 :  41432
    - 정확도: 0.7552, 정밀도: 0.6510, 재현율: 0.6808,    F1: 0.6655, AUC:0.8410
    - time : 8.08735203742981

- 이상치 제거하면 정확도는 상승, 재현율 하락
- 이상치 제거하지 않으면 정확도는 소폭 하락, 재현율은 상승
- 중요도 순으로 이상치 제거함수를 적용하고 재현율과 정밀도가 균형을 이룬 모델을 최종 선택함

In [24]:
coef = feature_importance()
high_coef_features = coef[:1].index 

/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [25]:
start = time.time()

# 이상치 제거함수 순차 적용
# 최적하이퍼파라미터 적용
x_train = pd.read_csv('./data/Smoking_raw/competition_format/x_train.csv')
x_test = pd.read_csv('./data/Smoking_raw/competition_format/x_test.csv')
y_train = pd.read_csv('./data/Smoking_raw/competition_format/y_train.csv')
y_test = pd.read_csv('./data/Smoking_raw/competition_format/y_test.csv')
# 전처리 - 불필요 컬럼 제거, 범주형 피처 레이블 인코딩, bmi 컬럼 생성, 정규화작업   
scaled = scaling(x_train, x_test,'logScaler')
x_train = pd.DataFrame(scaled[0])
x_test = pd.DataFrame(scaled[1])
y_train = preprocessing(y_train)
y_test = preprocessing(y_test)
    

# 가장 낮은 상관도를 보인 1개 컬럼 삭제
x_train.drop(coef[-2:].index,axis=1,inplace=True)
x_test.drop(coef[-2:].index,axis=1,inplace=True)
    
# 함수 사용해서 이상치 값 삭제
# 중요도가 높은 순으로 n개의 컬럼에 이상치 제거 함수 적용해보기
for k in high_coef_features:
    outlier_idx = get_outlier(df=x_train, column=k, weight=1.5)
    x_train.drop(outlier_idx, axis=0, inplace=True)
    y_train.drop(outlier_idx, axis=0, inplace=True)
for j in high_coef_features:
    outlier_idx = get_outlier(df=x_test, column=j, weight=1.5)
    x_test.drop(outlier_idx, axis=0, inplace=True)
    y_test.drop(outlier_idx, axis=0, inplace=True)

lr_clf = LogisticRegression(C=11, penalty='l1', solver='liblinear', max_iter=1000)
lr_clf.fit(x_train, y_train)
lr_pred = lr_clf.predict(x_test)
pred_prob = (lr_clf.predict_proba(x_test)[:,1])
# 평가
print('현재 컬럼 수 : ',len(x_train.columns),'이상치 제거함수 적용 수 : ',len(high_coef_features),' 행 갯수 : ',len(x_test.index))
get_clf_eval(y_test,lr_pred, pred_prob)
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

/var/folders/1n/3mts4mns2tq4k5ysdyjrwxmc0000gn/T/ipykernel_15178/3011367008.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = 0
/var/folders/1n/3mts4mns2tq4k5ysdyjrwxmc0000gn/T/ipykernel_15178/3011367008.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = 2
/var/folders/1n/3mts4mns2tq4k5ysdyjrwxmc0000gn/T/ipykernel_15178/3011367008.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = 1
/var/folders/1n/3mts4mns2

현재 컬럼 수 :  24 이상치 제거함수 적용 수 :  1  행 갯수 :  11139
오차 행렬
[[5572 1464]
 [1230 2873]]
정확도: 0.7581, 정밀도: 0.6624, 재현율: 0.7002,    F1: 0.6808, AUC:0.8433
time : 31.219161987304688
